# Bump test recent history

In [ ]:
# Times Square parameters

days_to_plot = 30


In [ ]:
import asyncio, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient
from lsst.ts.xml.enums.MTM1M3 import BumpTest

In [ ]:
def max_error(errors):
    return np.max([np.max(errors), np.max(errors * -1.0)])

def rms_error(times, errors):
    error = 0.0
    num = 0
    for i, t in enumerate(times):
        if (t > 3.0 and t < 4.0) or (t > 10.0 and t < 11.0):
            num += 1
            error += errors[i]**2
    if num == 0:
        return np.nan
    else:
        return np.sqrt(error / num)
        
async def calc_bumps_and_errors(bump, bt_result, follow):
    BUMP_TEST_DURATION = 14.0  # seconds
    measured_forces_times = []
    following_error_values = []
    t_starts = []
    results = bt_result[bt_result[bump] == BumpTest.TESTINGPOSITIVE]
    print(len(results))
    for bt_index in range(len(results)):
        t_start = Time(
            bt_result[bt_result[bump] == BumpTest.TESTINGPOSITIVE][
                "timestamp"
            ].values[bt_index]
            - 1.0,
            format="unix_tai",
            scale="tai",
        )
        t_starts.append(t_start.isot.split('.')[0])
        t_end = Time(
            t_start + TimeDelta(BUMP_TEST_DURATION, format="sec"),
            format="unix_tai",
            scale="tai",
        )
        print(f"Getting measured forces for {bump}, {bt_index}, {t_start.utc.isot}, {t_end.utc.isot}")
        measured_forces = await client.select_time_series(\
                    "lsst.sal.MTM1M3.forceActuatorData", \
                    [follow, "timestamp"], t_start.utc, t_end.utc)
        print(bump, bt_index, len(measured_forces))
        t0 = measured_forces["timestamp"].values[0]
        measured_forces["timestamp"] -= t0
    
        # It is easier/faster to work with arrays
        measured_forces_time = measured_forces["timestamp"].values
        measured_forces_times.append(measured_forces_time)
        following_error_value = measured_forces[follow].values
        following_error_values.append(following_error_value)

    times = []
    max_errors = []
    rms_errors = []
    print(len(measured_forces_times), "Starting time loop")
    for i in range(len(measured_forces_times)):
        print(i, t_starts[i])
        times.append(t_starts[i])
        max_errors.append(max_error(following_error_values[i]))
        rms_errors.append(rms_error(measured_forces_times[i], following_error_values[i]))
    print("Done, returning")
    return [times, rms_errors, max_errors]

In [ ]:
async def actuator_error(client, fa_id, bt_results):    
    # Grab the Force Actuator Data from its ID
    fa_data = force_actuator_from_id(fa_id)
    bt_result = bt_results[bt_results["actuatorId"] == fa_id]
    
    # First the primary forces
    bump = f"primaryTest{fa_data.index}"
    follow = f"primaryCylinderFollowingError{fa_data.index}"
    [ptimes, prms_errors, pmax_errors] = \
        await calc_bumps_and_errors(bump, bt_result, follow)
    print(f"Finished primary forces for {fa_id}")
    # Now the secondary  forces  
    if fa_data.actuator_type.name == "DAA":
        bump = f"secondaryTest{fa_data.s_index}"
        follow = f"secondaryCylinderFollowingError{fa_data.s_index}"
            
        [stimes, srms_errors, smax_errors] = \
            await calc_bumps_and_errors(bump, bt_result, follow)
        print(f"Finished secondary forces for {fa_id}")
    else:
        stimes = []; srms_errors = []; smax_errors = []
        print("No secondary")

    return [ptimes, prms_errors, pmax_errors, stimes, srms_errors, smax_errors]


In [ ]:
end = Time.now()
start = end - TimeDelta(days_to_plot, format='jd')
client = EfdClient('usdf_efd')
query = f'''SELECT * FROM "lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus" WHERE time > '{start.isot}Z' AND time < '{end.isot}Z' '''
chunks = await client.influx_client.query(query, chunked=True, chunk_size=1000)
bumps = pd.concat([chunk async for chunk in chunks])

ids = []
for index in range(len(FATable)):
    id = FATable[index].actuator_id
    ids.append(id)


In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,2, figsize=(10,10))
plt.suptitle("Actuator Recent History\nNot plotted -> All green", fontsize=18)
plot_list = [[0,0,"Primary RMS Errors", 0, 1, 5, 10],
             [1,0,"Primary Max Errors", 0, 2, 100, 200],
             [0,1,"Secondary RMS Errors", 3, 4, 7.0, 14.0],
             [1,1,"Secondary Max Errors", 3, 5, 140, 280]]
ns = np.zeros(4) # Number of bad ids per plot
bad_ids = [[],[],[],[]] # List of bad_ids per plot
min_times = np.ones(4) * 1.0E12 # Min/Max times in unix_tai
max_times = np.zeros(4) # Min/Max times in unix_tai
min_time_list = [[],[],[],[]] # Min/Max times in isot
max_time_list = [[],[],[],[]] # Min/Max times in isot
for id in ids:
    data = await actuator_error(client, id, bumps)
    for m, [xplot, yplot, title, time_index, error_index, yellow_limit, red_limit] in enumerate(plot_list):
        errors = data[error_index]
        if len(errors) > 0:
            max = np.max(errors)
        else:
            max = 0.0
        if max > yellow_limit:
            bad_ids[m].append(id)
        else:
            continue

        times = data[time_index]
        actual_times_red = []
        actual_times_yellow = []
        actual_times_green = []
        yaxis_red = []
        yaxis_yellow = []
        yaxis_green = []
    
        for i, time in enumerate(times):
            act_time = Time(time, format='isot').unix_tai
            if act_time < min_times[m]:
                min_times[m] = act_time
                min_time_list[m] = time
            if act_time > max_times[m]:
                max_times[m] = act_time
                max_time_list[m] = time
            
            if errors[i] > red_limit:
                actual_times_red.append(act_time)
                yaxis_red.append(ns[m])
            elif errors[i] > yellow_limit:
                actual_times_yellow.append(act_time)
                yaxis_yellow.append(ns[m])
            else:
                actual_times_green.append(act_time)
                yaxis_green.append(ns[m])

        axs[xplot][yplot].set_title(title)
        axs[xplot][yplot].scatter(actual_times_green, yaxis_green, color='green')
        axs[xplot][yplot].scatter(actual_times_yellow, yaxis_yellow, color='gold')
        axs[xplot][yplot].scatter(actual_times_red, yaxis_red, color='red')
        ns[m] += 1
    print(f"Finished ID {id}")
for m, [xplot, yplot, title, time_index, error_index, yellow_limit, red_limit] in enumerate(plot_list):
    axs[xplot][yplot].set_yticks(list(range(len(bad_ids[m]))), bad_ids[m])
    axs[xplot][yplot].set_xticks([min_times[m], max_times[m]],[min_time_list[m], max_time_list[m]], rotation=10)
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Actuator_Trends_11Oct24.pdf")
#plt.show()

In [ ]:
t0 = Time.now()
start = Time('2024-09-19T12:24:07.288')
end = Time('2024-09-19T12:24:21.288')
follow = 'primaryCylinderFollowingError0'
measured_forces = await client.select_time_series(\
            "lsst.sal.MTM1M3.forceActuatorData", \
            [follow, "timestamp"], start, end)
t1 = Time.now()

print(len(measured_forces), f"Time = {(t1 - t0).value * 86400 * 1000} ms")

In [ ]:
start = Time('2024-09-20T21:06:12.782')
end = Time('2024-09-20T21:06:26.782')
follow = 'primaryCylinderFollowingError0'
measured_forces = await client.select_time_series(\
            "lsst.sal.MTM1M3.forceActuatorData", \
            [follow, "timestamp"], start, end)
print(len(measured_forces))

In [ ]:
start = Time.now()

In [ ]:
end = Time.now()

In [ ]:
(end - start).value * 86400